In [3]:
import os
import sys
import numpy as np
from sklearn.metrics import hamming_loss, accuracy_score, precision_score, recall_score, f1_score

In [4]:
NPY_FILE_PATH = "../npy_files/13Jun_multilabel"

filenames = os.listdir(path=NPY_FILE_PATH)
filenames = sorted(filenames, key=lambda x: x.split('_')[-1])
filenames = list(set([filename.split('_')[2] for filename in filenames]))

In [5]:
def hamming_score(y_true, y_pred):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    
    return np.mean(acc_list)

In [6]:
filename = filenames[0]
filenames

['worthy-field-274.npy']

In [7]:
eval_labels_data = np.load(f'{NPY_FILE_PATH}/eval_labels_{filename}', allow_pickle=True)

In [8]:
eval_labels_data[0][1][0]

[0.0, 0.0, 0.0, 1.0]

In [21]:
for filename in filenames:
        run_name = filename.split('.')[0]
        print(f"The metrics for {run_name} run are:")
        labels_data = np.load(f'{NPY_FILE_PATH}/test_labels_{filename}', allow_pickle=True)
        preds_data = np.load(f'{NPY_FILE_PATH}/test_preds_{filename}', allow_pickle=True)
        eval_labels_data = np.load(f'{NPY_FILE_PATH}/eval_labels_{filename}', allow_pickle=True)
        eval_preds_data = np.load(f'{NPY_FILE_PATH}/eval_preds_{filename}', allow_pickle=True)
        labels = []
        preds = []
        eval_labels = []
        eval_preds = []
        for i in range(len(labels_data)):
                for j in range(len(labels_data[i][1])):
                        labels.append(labels_data[i][1][j])
                        preds.append(preds_data[i][1][j])
        for i in range(len(eval_labels_data)):
                for j in range(len(eval_labels_data[i][1])):
                        eval_labels.append(eval_labels_data[i][1][j])
                        eval_preds.append(eval_preds_data[i][1][j])
        # for i in range(len(eval_labels_data)):
        # for j in range(len(eval_labels_data[i])):
        #         eval_labels.append(eval_labels_data[i][j][:5])
        #         eval_preds.append(eval_preds_data[i][j][:5])
        labels = np.array(labels)
        preds = np.array(preds)
        eval_labels = np.array(eval_labels)
        eval_preds = np.array(eval_preds)
        print(labels.shape, preds.shape, eval_labels.shape, eval_preds.shape)
        max_hamming_score = 0
        min_hamming_loss = 1
        max_thres = 0
        for thresh in np.linspace(0, 1, 11):
                copy_preds = eval_preds.copy()
                copy_preds[copy_preds >= thresh] = 1
                copy_preds[copy_preds < thresh] = 0
                print("Thresh:", thresh, "\tHamming Loss:", hamming_loss(copy_preds, eval_labels), "\tHamming Score:", hamming_score(copy_preds, eval_labels))
                if max_hamming_score > hamming_score(eval_labels, copy_preds):
                        max_hamming_score = hamming_score(eval_labels, copy_preds)
                        max_thres = thresh
                        print(max_hamming_score)
                        # min_hamming_loss = hamming_loss(eval_labels, copy_preds)
        copy_preds = preds.copy()
        copy_preds[copy_preds >= max_thres] = 1
        copy_preds[copy_preds < max_thres] = 0                
        
        print("Thresh:", max_thres, "\tHamming Loss:", hamming_loss(copy_preds, labels), "\tHamming Score:", hamming_score(copy_preds, labels))
        preds[preds >= 0.5] = 1
        preds[preds < 0.5] = 0
        
        mapping = {0: "Organisation", 1: "Location", 2: "Individual", 3: "Community", 4: "None"}
        for i in range(0, 4):
                print(f"{mapping[i]} Precision: {precision_score(labels[:, i], preds[:, i])} Recall: {recall_score(labels[:, i], preds[:, i])} F1 Score: {f1_score(labels[:, i], preds[:, i])}")
        print()

The metrics for worthy-field-274 run are:
(287, 4) (287, 4) (262, 4) (262, 4)
Thresh: 0.0 	Hamming Loss: 0.43416030534351147 	Hamming Score: 0.3435114503816794
Thresh: 0.1 	Hamming Loss: 0.41603053435114506 	Hamming Score: 0.33587786259541985
Thresh: 0.2 	Hamming Loss: 0.39599236641221375 	Hamming Score: 0.3136132315521628
Thresh: 0.30000000000000004 	Hamming Loss: 0.3797709923664122 	Hamming Score: 0.2862595419847328
Thresh: 0.4 	Hamming Loss: 0.3597328244274809 	Hamming Score: 0.24681933842239187
Thresh: 0.5 	Hamming Loss: 0.3101145038167939 	Hamming Score: 0.23091603053435114
Thresh: 0.6000000000000001 	Hamming Loss: 0.29866412213740456 	Hamming Score: 0.1895674300254453
Thresh: 0.7000000000000001 	Hamming Loss: 0.29961832061068705 	Hamming Score: 0.1628498727735369
Thresh: 0.8 	Hamming Loss: 0.29866412213740456 	Hamming Score: 0.1450381679389313
Thresh: 0.9 	Hamming Loss: 0.2967557251908397 	Hamming Score: 0.1342239185750636
Thresh: 1.0 	Hamming Loss: 0.2948473282442748 	Hamming Sc

In [ ]:
preds[preds >= 0.5] = 1
preds[preds < 0.5] = 0

In [ ]:
mapping = {0: "Organisation", 1: "Location", 2: "Individual", 3: "Community", 4: "None"}
for i in range(0, 5):
    print(f"{mapping[i]} Precision: {precision_score(labels[:, i], preds[:, i])} Recall: {recall_score(labels[:, i], preds[:, i])} F1 Score: {f1_score(labels[:, i], preds[:, i])}")

In [ ]:
preds

In [ ]:
idx = []
for i in range(len(preds)):
    if (preds[i] != labels[i]).any():
        idx.append(i)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/hate_towards_whom_aug/test.csv')
df = df.iloc[idx]

In [ ]:
[label for label in labels[idx].tolist()]

In [ ]:
df['prediction'] = [",".join([mapping[i] for i, _pred in enumerate(pred) if _pred == 1]) for pred in preds[idx].tolist()]

In [ ]:
df.tail(50)